#### Imports & Config

In [1]:
import requests
import pandas as pd
import json
import os
from typing import Dict, List

#### API Configuration

In [2]:
API_KEY = "qc777hZbWwDpCNAEYG2rkPmRA4cjbfbzxySfKSAD"

BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en"

HEADERS = {
    "accept": "application/json"
}

#### Test API & Fetch Competitions Data

In [3]:
import requests
from pprint import pprint

# Endpoint for competitions
endpoint = f"{BASE_URL}/competitions.json"

# Query parameters
params = {
    "api_key": API_KEY
}

try:
    response = requests.get(endpoint, headers=HEADERS, params=params, timeout=30)
    response.raise_for_status()  # Raises HTTPError for bad responses (4xx, 5xx)

    data = response.json()

    print("✅ API Connection Successful")
    print(f"Status Code: {response.status_code}")
    print(f"Top-level keys in response: {list(data.keys())}")

    # Basic sanity check
    if "competitions" in data and isinstance(data["competitions"], list):
        print(f"Total competitions received: {len(data['competitions'])}")
        print("\nSample competition record:")
        pprint(data["competitions"][0])
    else:
        print("⚠️ Unexpected response structure")

except requests.exceptions.HTTPError as http_err:
    print(f"❌ HTTP error occurred: {http_err}")

except requests.exceptions.RequestException as req_err:
    print(f"❌ Request error occurred: {req_err}")

except ValueError:
    print("❌ Failed to parse JSON response")

✅ API Connection Successful
Status Code: 200
Top-level keys in response: ['generated_at', 'competitions']
Total competitions received: 6437

Sample competition record:
{'category': {'id': 'sr:category:181', 'name': 'Hopman Cup'},
 'gender': 'mixed',
 'id': 'sr:competition:620',
 'name': 'Hopman Cup',
 'type': 'mixed'}


#### Save Raw API Response Locally

In [4]:
import json
import os
from datetime import datetime

# Create directory if not exists
os.makedirs("data/raw", exist_ok=True)

# File name with timestamp
file_path = f"data/raw/competitions_raw_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

# Save response JSON
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print(f"Raw competition data saved successfully at: {file_path}")

Raw competition data saved successfully at: data/raw/competitions_raw_20251224_144104.json


### Data Extraction + Sanity Checks (Competitions & Categories)
#### Extract Competitions & Categories

#### Save Extracted Data

In [10]:
import os
import json
import pandas as pd

# ===============================
# Project Paths (NEW ROOT)
# ===============================
PROJECT_ROOT = r"C:\Users\susen\Desktop\Tennis Game"

RAW_DIR = os.path.join(PROJECT_ROOT, "data", "raw")
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

# ===============================
# Raw File Path
# (copy the file here if not already)
# ===============================
RAW_FILE_NAME = "competitions_raw_20251224_144104.json"
RAW_FILE_PATH = os.path.join(RAW_DIR, RAW_FILE_NAME)

if not os.path.exists(RAW_FILE_PATH):
    raise FileNotFoundError(
        f"❌ Raw file not found at: {RAW_FILE_PATH}\n"
        f"➡️ Please copy the raw JSON into data/raw/ folder"
    )

print(f"📂 Loading raw file: {RAW_FILE_PATH}")

# ===============================
# Load Raw JSON
# ===============================
with open(RAW_FILE_PATH, "r", encoding="utf-8") as file:
    raw_data = json.load(file)

competitions = raw_data.get("competitions", [])

print(f"✅ Total competitions loaded: {len(competitions)}")

# ===============================
# Extract Categories (deduplicated)
# ===============================
categories_records = {}

for comp in competitions:
    cat = comp.get("category", {})
    if cat:
        categories_records[cat["id"]] = {
            "category_id": cat["id"],
            "category_name": cat["name"]
        }

df_categories = pd.DataFrame(categories_records.values())

# ===============================
# Extract Competitions
# ===============================
competition_rows = []

for comp in competitions:
    competition_rows.append({
        "competition_id": comp.get("id"),
        "competition_name": comp.get("name"),
        "parent_id": comp.get("parent_id"),
        "type": comp.get("type"),
        "gender": comp.get("gender"),
        "category_id": comp.get("category", {}).get("id")
    })

df_competitions = pd.DataFrame(competition_rows)

# ===============================
# Final Sanity Checks
# ===============================
print("\n📊 Categories shape:", df_categories.shape)
print("📊 Competitions shape:", df_competitions.shape)

print("\n🔎 Missing values (Competitions):")
print(df_competitions.isna().sum())

# ===============================
# Save Processed Files
# ===============================
categories_path = os.path.join(PROCESSED_DIR, "categories.csv")
competitions_path = os.path.join(PROCESSED_DIR, "competitions.csv")

df_categories.to_csv(categories_path, index=False)
df_competitions.to_csv(competitions_path, index=False)

print("\n✅ Processed datasets saved successfully")
print("📁 Categories:", categories_path)
print("📁 Competitions:", competitions_path)

📂 Loading raw file: C:\Users\susen\Desktop\Tennis Game\data\raw\competitions_raw_20251224_144104.json
✅ Total competitions loaded: 6437

📊 Categories shape: (18, 2)
📊 Competitions shape: (6437, 6)

🔎 Missing values (Competitions):
competition_id        0
competition_name      0
parent_id           496
type                  0
gender                6
category_id           0
dtype: int64

✅ Processed datasets saved successfully
📁 Categories: C:\Users\susen\Desktop\Tennis Game\data\processed\categories.csv
📁 Competitions: C:\Users\susen\Desktop\Tennis Game\data\processed\competitions.csv


#### Final Pre-SQL Validation

In [11]:
import pandas as pd
import os

# Base project path
BASE_PATH = r"C:\Users\susen\Desktop\Tennis Game"

processed_dir = os.path.join(BASE_PATH, "data", "processed")

categories_path = os.path.join(processed_dir, "categories.csv")
competitions_path = os.path.join(processed_dir, "competitions.csv")

# Load processed datasets
df_categories = pd.read_csv(categories_path)
df_competitions = pd.read_csv(competitions_path)

print("✅ Files loaded successfully")

# ---------- Sanity Checks ----------

print("\n📊 Categories Info")
display(df_categories.info())

print("\n📊 Competitions Info")
display(df_competitions.info())

# Check duplicate primary keys
print("\n🔎 Duplicate Checks")
print("Duplicate Categories:", df_categories['category_id'].duplicated().sum())
print("Duplicate Competitions:", df_competitions['competition_id'].duplicated().sum())

# Foreign key integrity check
missing_categories = set(df_competitions['category_id']) - set(df_categories['category_id'])
print("\n🔗 FK Integrity Check")
print("Missing category references:", len(missing_categories))

if len(missing_categories) == 0:
    print("✅ Foreign key integrity PASSED")
else:
    print("❌ Foreign key integrity FAILED")

# Quick preview
display(df_competitions.head())

✅ Files loaded successfully

📊 Categories Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category_id    18 non-null     object
 1   category_name  18 non-null     object
dtypes: object(2)
memory usage: 420.0+ bytes


None


📊 Competitions Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6437 entries, 0 to 6436
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   competition_id    6437 non-null   object
 1   competition_name  6437 non-null   object
 2   parent_id         5941 non-null   object
 3   type              6437 non-null   object
 4   gender            6431 non-null   object
 5   category_id       6437 non-null   object
dtypes: object(6)
memory usage: 301.9+ KB


None


🔎 Duplicate Checks
Duplicate Categories: 0
Duplicate Competitions: 0

🔗 FK Integrity Check
Missing category references: 0
✅ Foreign key integrity PASSED


,competition_id,competition_name,parent_id,type,gender,category_id
0,sr:competition:620,Hopman Cup,NaN,mixed,mixed,sr:category:181
1,sr:competition:660,World Team Cup,NaN,mixed,men,sr:category:3
2,sr:competition:990,ATP Challenger Tour Finals,sr:competition:6239,singles,men,sr:category:72
3,sr:competition:1207,Championship International Series,NaN,singles,women,sr:category:6
4,sr:competition:2100,Davis Cup,NaN,mixed,men,sr:category:76


#### API Endpoint: https://api.sportradar.com/tennis/trial/v3/en/complexes.json
#### Call Complexes API

In [14]:
import requests
import json
import pandas as pd
from datetime import datetime
import os

# API settings
API_KEY = "qc777hZbWwDpCNAEYG2rkPmRA4cjbfbzxySfKSAD"
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en"
endpoint = "complexes.json"

# Make Request
response = requests.get(f"{BASE_URL}/{endpoint}?api_key={API_KEY}")

if response.status_code != 200:
    print("❌ API Failed:", response.status_code, response.text)
else:
    data = response.json()
    print("✅ Loaded complexes:", len(data['complexes']))

    # Save raw JSON
    raw_folder = r"C:\Users\susen\Desktop\Tennis Game\data\raw"
    os.makedirs(raw_folder, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    raw_path = os.path.join(raw_folder, f"complexes_raw_{timestamp}.json")
    with open(raw_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    print("📂 Raw saved at:", raw_path)

    # Parse complexes
    complexes_list = []
    venues_list = []

    for c in data["complexes"]:
        complexes_list.append({
            "complex_id": c["id"],
            "complex_name": c.get("name")
        })

        for v in c.get("venues", []):
            venues_list.append({
                "venue_id": v.get("id"),
                "venue_name": v.get("name"),
                "city_name": v.get("city_name"),
                "country_name": v.get("country_name"),
                "country_code": v.get("country_code"),
                "timezone": v.get("timezone"),
                "complex_id": c["id"]
            })

    # Build DataFrames
    df_complexes = pd.DataFrame(complexes_list)
    df_venues = pd.DataFrame(venues_list)

    print("\n📊 Complexes DF Info")
    print(df_complexes.info())
    print("\n📊 Venues DF Info")
    print(df_venues.info())

    # Save processed CSV
    processed_folder = r"C:\Users\susen\Desktop\Tennis Game\data\processed"
    os.makedirs(processed_folder, exist_ok=True)

    complexes_csv = os.path.join(processed_folder, "complexes.csv")
    venues_csv = os.path.join(processed_folder, "venues.csv")

    df_complexes.to_csv(complexes_csv, index=False)
    df_venues.to_csv(venues_csv, index=False)

    print("\n✅ CSVs saved successfully:")
    print("📁 Complexes:", complexes_csv)
    print("📁 Venues:", venues_csv)

✅ Loaded complexes: 725
📂 Raw saved at: C:\Users\susen\Desktop\Tennis Game\data\raw\complexes_raw_20251226_222501.json

📊 Complexes DF Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   complex_id    725 non-null    object
 1   complex_name  725 non-null    object
dtypes: object(2)
memory usage: 11.5+ KB
None

📊 Venues DF Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3802 entries, 0 to 3801
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   venue_id      3802 non-null   object
 1   venue_name    3802 non-null   object
 2   city_name     3802 non-null   object
 3   country_name  3802 non-null   object
 4   country_code  3802 non-null   object
 5   timezone      3802 non-null   object
 6   complex_id    3802 non-null   object
dtypes: object(7)
memory usage: 208.1+ K

#### 2. DOUBLES COMPETITOR RANKINGS API Extraction

In [15]:
import requests
import json
import pandas as pd
from datetime import datetime
import os

API_KEY = "qc777hZbWwDpCNAEYG2rkPmRA4cjbfbzxySfKSAD"
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en"
endpoint = "doubles-competitor-rankings.json"

response = requests.get(f"{BASE_URL}/{endpoint}?api_key={API_KEY}")

if response.status_code != 200:
    print("❌ API Failed:", response.status_code, response.text)
else:
    data = response.json()
    print("✅ Rankings loaded")

    # Save RAW JSON
    raw_dir = r"C:\Users\susen\Desktop\Tennis Game\data\raw"
    os.makedirs(raw_dir, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    raw_path = os.path.join(raw_dir, f"doubles_rankings_raw_{timestamp}.json")

    with open(raw_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

    print("📂 Raw file saved:", raw_path)

❌ API Failed: 404 {"generated_at":"2025-12-27T05:42:57+00:00","message":"Invalid route."}


#### Create a mock file due to ❌ API Failed: 404 {"generated_at":"2025-12-27T05:42:57+00:00","message":"Invalid route."}

In [16]:
import json
import os
from datetime import datetime

# Mock API-like data
mock_data = {
    "generated_at": "2025-12-27T05:50:00+00:00",
    "rankings": [
        {
            "rank": 1,
            "movement": 0,
            "points": 9200,
            "competitions_played": 18,
            "competitor": {
                "id": "sr:competitor:101",
                "name": "Nikola Mektic",
                "country": "Croatia",
                "country_code": "HRV",
                "abbreviation": "MEK"
            }
        },
        {
            "rank": 2,
            "movement": 1,
            "points": 8900,
            "competitions_played": 17,
            "competitor": {
                "id": "sr:competitor:102",
                "name": "Mate Pavic",
                "country": "Croatia",
                "country_code": "HRV",
                "abbreviation": "PAV"
            }
        },
        {
            "rank": 3,
            "movement": -1,
            "points": 8500,
            "competitions_played": 16,
            "competitor": {
                "id": "sr:competitor:103",
                "name": "Rajeev Ram",
                "country": "USA",
                "country_code": "USA",
                "abbreviation": "RAM"
            }
        }
    ]
}

# Save raw mock JSON
raw_dir = r"C:\Users\susen\Desktop\Tennis Game\data\raw"
os.makedirs(raw_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
raw_path = os.path.join(raw_dir, f"doubles_rankings_mock_{timestamp}.json")

with open(raw_path, "w", encoding="utf-8") as f:
    json.dump(mock_data, f, indent=2)

print("✅ Mock raw rankings JSON saved at:")
print(raw_path)

✅ Mock raw rankings JSON saved at:
C:\Users\susen\Desktop\Tennis Game\data\raw\doubles_rankings_mock_20251227_114809.json


#### Parse MOCK JSON → DataFrames

In [17]:
import pandas as pd

# Load mock JSON
with open(raw_path, "r", encoding="utf-8") as f:
    data = json.load(f)

competitors = []
rankings = []

for r in data["rankings"]:
    comp = r["competitor"]

    competitors.append({
        "competitor_id": comp["id"],
        "name": comp["name"],
        "country": comp["country"],
        "country_code": comp["country_code"],
        "abbreviation": comp["abbreviation"]
    })

    rankings.append({
        "rank": r["rank"],
        "movement": r["movement"],
        "points": r["points"],
        "competitions_played": r["competitions_played"],
        "competitor_id": comp["id"]
    })

df_competitors = pd.DataFrame(competitors)
df_rankings = pd.DataFrame(rankings)

print("📊 Competitors Info")
print(df_competitors.info())

print("\n📊 Rankings Info")
print(df_rankings.info())

📊 Competitors Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   competitor_id  3 non-null      object
 1   name           3 non-null      object
 2   country        3 non-null      object
 3   country_code   3 non-null      object
 4   abbreviation   3 non-null      object
dtypes: object(5)
memory usage: 252.0+ bytes
None

📊 Rankings Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   rank                 3 non-null      int64 
 1   movement             3 non-null      int64 
 2   points               3 non-null      int64 
 3   competitions_played  3 non-null      int64 
 4   competitor_id        3 non-null      object
dtypes: int64(4), object(1)
memory usage: 252.0+ bytes
None


#### Save Processed CSVs

In [18]:
processed_dir = r"C:\Users\susen\Desktop\Tennis Game\data\processed"
os.makedirs(processed_dir, exist_ok=True)

competitors_csv = os.path.join(processed_dir, "competitors.csv")
rankings_csv = os.path.join(processed_dir, "competitor_rankings.csv")

df_competitors.to_csv(competitors_csv, index=False)
df_rankings.to_csv(rankings_csv, index=False)

print("✅ Processed CSVs saved:")
print(competitors_csv)
print(rankings_csv)

✅ Processed CSVs saved:
C:\Users\susen\Desktop\Tennis Game\data\processed\competitors.csv
C:\Users\susen\Desktop\Tennis Game\data\processed\competitor_rankings.csv
